In [72]:
import pandas
import numpy as np
import connect_net
import torch

In [57]:
df = pandas.read_csv("connect-4.data", header=None)
print(len(df))
cols = [f"a{i}" for i in range(1, 7)] \
    + [f"b{i}" for i in range(1, 7)] \
    + [f"c{i}" for i in range(1, 7)] \
    + [f"d{i}" for i in range(1, 7)] \
    + [f"e{i}" for i in range(1, 7)] \
    + [f"f{i}" for i in range(1, 7)] \
    + [f"g{i}" for i in range(1, 7)]
df = df.rename(columns={i: c for i, c in enumerate(cols)})
df = df.rename(columns={42: "result"})
for c in cols:
    df[c] = df[c].map({"b": 0, "x": 1, "o": -1})
df["result"] = df["result"].map({"win": 1, "draw": 0, "loss": -1})
df.head(3)

67557


,a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,...,f4,f5,f6,g1,g2,g3,g4,g5,g6,result
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,-1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [46]:
np.flip(np.reshape(np.array(list(range(1, 43))), (7, 6)).T, axis=0)

array([[ 6, 12, 18, 24, 30, 36, 42],
       [ 5, 11, 17, 23, 29, 35, 41],
       [ 4, 10, 16, 22, 28, 34, 40],
       [ 3,  9, 15, 21, 27, 33, 39],
       [ 2,  8, 14, 20, 26, 32, 38],
       [ 1,  7, 13, 19, 25, 31, 37]])

In [ ]:
vals = []
for i in range(len(df)):
    if i % 10000 == 0:
        print(i)
    vals.append(np.flip(np.reshape(df.iloc[i][cols].values, (7, 6)).T, axis=0))

In [64]:
vals_stack = np.stack(vals)
res_vals = df["result"].values

In [68]:
i = 3
print(vals_stack[i])
print(res_vals[i])

[[ 0  0  0 -1  0  0  0]
 [ 0  0  0  1  0  0  0]
 [ 0  0  0 -1  0  0  0]
 [ 0  0  0  1  0  0  0]
 [ 0  0  0 -1  0  0  0]
 [ 0  0  1  1  0 -1  0]]
1


In [82]:
input_vals = torch.Tensor(vals_stack).unsqueeze(1)
output_vals = torch.Tensor(df["result"].values)
input_vals[:100].shape

torch.Size([100, 1, 6, 7])

In [88]:
nnet = connect_net.ConnectNet()
optimizer = torch.optim.Adam(nnet.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss()


In [181]:

v, proba = nnet(input_vals[100:200])

loss = loss_fn(v, output_vals[100:200])
print(loss)
print(v.shape)
print(proba.shape)
optimizer.zero_grad()
loss.backward()
optimizer.step()

tensor(0.8311, grad_fn=<MseLossBackward0>)
torch.Size([100])
torch.Size([100, 7])
